In [9]:
#!pip install pqdm plotnine joblib pandas

In [2]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import hashlib

import pandas as pd
from pqdm.threads import pqdm
from plotnine import *

import urllib.request
from joblib import Parallel, delayed

import seaborn as sns

In [3]:
plt.rcParams['figure.figsize'] = 20, 10

In [4]:
!mkdir -p images # crea carpeta de destino

In [5]:
data = pd.read_json('formatted_data/ExpectedEmotion.json')

In [6]:
data['imgur_url'] = data['imgur'].apply(lambda x: x['url'])
data['imgur_url_sha256'] = data['imgur_url'].apply(str.encode).apply(hashlib.sha256).apply(lambda x: x.hexdigest())

In [7]:
destination_path_root = './images/' 

In [8]:
data['imgur_destination'] = destination_path_root + data['imgur_url_sha256'] + '.' + data['filet']

In [8]:
# Las descargaremos con su sha256(url) como identificador.

In [9]:
to_download = data[['imgur_url','imgur_url_sha256','imgur_destination']].drop_duplicates()
to_download['downloaded'] = to_download['imgur_destination'].apply(os.path.exists)
to_download = to_download[~to_download['downloaded']].copy()

In [10]:
len(data)

1057748

In [11]:
len(to_download)

968705

# Downloading data

In [12]:
def download_data(_id, url, dest_path):
    import urllib.request
    
    urllib.request.urlretrieve(url, dest_path)

In [13]:
download_tuples = list(to_download[['imgur_url_sha256','imgur_url','imgur_destination']].itertuples(index=None, name=None))

In [14]:
download_tuples[0]

('1ccbd4dbf5729d492a9e094c25ffc5509107a604ad6904d2750e8c06695ebbb7',
 'http://farm1.staticflickr.com/1/24489_bc0c04ae85.jpg',
 './images/1ccbd4dbf5729d492a9e094c25ffc5509107a604ad6904d2750e8c06695ebbb7.jpg')

In [15]:
result = pqdm(download_tuples, lambda download_tuple: download_data(*download_tuple), n_jobs=12)

SUBMITTING | :   0%|          | 0/968705 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/968705 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [78]:
download_tuples[-1000]

('e702f2e75e4d6b541d531cd51ba658b5091ea80cd375533ca327ffab209d88c9',
 'http://farm4.staticflickr.com/3318/3246938460_50fb087160_z.jpg',
 './images/e702f2e75e4d6b541d531cd51ba658b5091ea80cd375533ca327ffab209d88c9.jpg')

In [77]:
result[-1000]

urllib.error.URLError(socket.gaierror(8,
                                      'nodename nor servname provided, or not known'))